In [8]:
import os
from PIL import Image
from pathlib import Path
import pyguetzli as pg

In [11]:
def show_file_size(size_in_bytes , format='auto'):
    unit = {'gb' : 1024 ** 3,
            'mb' : 1024 ** 2,
            'kb' : 1024 ** 1,
            'b' : 1024 ** 0}
    
    for key , val in unit.items():
        cal_val = size_in_bytes / val
        if(cal_val >= 1):
            return [cal_val , key]

print(show_file_size(1930736))

[1.8412933349609375, 'mb']


In [23]:
def compress_jpg(src_file_path , target_file_path):
    optimized_bytes = None
    try:
        with open(src_file_path,'rb') as image:
            optimized_bytes = pg.process_jpeg_bytes(image.read())

        with open(target_file_path,'wb') as image:
            image.write(optimized_bytes)
            return True
    except ValueError as e:
        print(f'[Error] : {e}')
        return False

In [24]:
def compress_png(src_file_path , target_file_path):
    optimized_bytes = None
    try:
        with Image.open(src_file_path) as image:
            optimized_bytes = pg.process_pil_image(image)
        with open(target_file_path , 'wb') as image:
            image.write(optimized_bytes)
            
    except ValueError as e:
        print(f'[Error] : {e}')
        return False

In [25]:
def batch_compress(directory , extra=''):
    if not os.path.isdir(directory):
        print(f'[Error] : `{directory}` is not a directory')
        return

    files = os.listdir(directory)
    file_counter = 1
    total_file_count = len(files)
    total_file_size = 0
    total_converted_file_size = 0
    
    for file in os.listdir(directory):
        src_file_path = os.path.join(directory,file)
        file_name , file_extention = file.split('.')
        file_extention = '.' + file_extention
        target_file_path = os.path.join(directory,file_name+extra+'.jpeg')

        print(f'[{file_counter} / {total_file_count}] File : {file_name+file_extention}')
        if file_extention in ['.jpg','.jpeg']:
            print('Compressing...')
            src_file_size = show_file_size(os.stat(src_file_path).st_size)
            if compress_jpg(src_file_path , target_file_path) :
                target_file_size = show_file_size(os.stat(target_file_path).st_size)
                total_file_size += src_file_size[0]
                total_converted_file_size += target_file_size[0]
                print(f'Done [{round(src_file_size[0],2)}{src_file_size[1].upper()} -> {round(target_file_size[0],2)}{target_file_size[1].upper()}]')
        elif file_extention == '.png':
            print('Compressing...')
            src_file_size = show_file_size(os.stat(src_file_path).st_size)
            if compress_png(src_file_path , target_file_path) :
                target_file_size = show_file_size(os.stat(target_file_path).st_size)
                total_file_size += src_file_size[0]
                total_converted_file_size += target_file_size[0]
                print(f'Done [{round(src_file_size[0],2)}{src_file_size[1].upper()} -> {round(target_file_size[0],2)}{target_file_size[1].upper()}]')
        else:
            print('File is not a valid image')

        file_counter += 1

In [26]:
#compress_jpg(Path(r'C:\Users\Rohan\Coding\Automation\Gutezli Photo Compressor\test_image.jpg') , Path(r'C:\Users\Rohan\Coding\Automation\Gutezli Photo Compressor\test_image_compressed.jpg'))
batch_compress(Path(r'C:\Users\Rohan\Coding\Automation\Gutezli Photo Compressor') , '_compressed')

[1 / 4] File : dev.ipynb
File is not a valid image
[2 / 4] File : main.py
File is not a valid image
[3 / 4] File : test_image.jpg
Compressing...


Done [318.83KB -> 92.28KB]
[4 / 4] File : test_image.png
Compressing...


In [3]:
print(os.stat(Path(r'C:\Users\Rohan\Coding\Automation\Gutezli Photo Compressor\test_image.jpg')))
print(os.stat(Path(r'C:\Users\Rohan\Coding\Automation\Gutezli Photo Compressor\test_image_compressed.jpg')))

os.stat_result(st_mode=33206, st_ino=11258999068633667, st_dev=5808752741427034020, st_nlink=1, st_uid=0, st_gid=0, st_size=326477, st_atime=1705468255, st_mtime=1695018782, st_ctime=1705467594)
os.stat_result(st_mode=33206, st_ino=5910974511349357, st_dev=5808752741427034020, st_nlink=1, st_uid=0, st_gid=0, st_size=94490, st_atime=1705468229, st_mtime=1705467706, st_ctime=1705467706)


In [9]:
def get_image_info(image_path):
    try:
        with Image.open(image_path) as image:
            # Get basic image information
            mode = image.mode
            size = image.size
            format = image.format

            # Get detailed image information from Exif metadata
            exif_info = image._getexif()

            return mode, size, format, exif_info

    except Exception as e:
        print(f'Error while processing image: {e}')
        return None

# Example usage
image_path = Path(r'C:\Users\Rohan\Coding\Automation\Gutezli Photo Compressor\test_image.jpg')
image_info = get_image_info(image_path)

if image_info:
    mode, size, format, exif_info = image_info

    print(f"Image Mode: {mode}")
    print(f"Image Size: {size[0]} x {size[1]} pixels")
    print(f"Image Format: {format}")

    if exif_info:
        for tag, value in exif_info.items():
            tag_name = ExifTags.TAGS.get(tag, tag)
            print(f"{tag_name}: {value}")
    else:
        print("No Exif information available.")
else:
    print("Unable to retrieve image information.")


Image Mode: RGB
Image Size: 608 x 739 pixels
Image Format: JPEG
No Exif information available.
